In [2]:
import os
import json
from src.dialogue_tools import create_rag, generate_answers, generate_follow_up_questions, cross_reference, aggregate_final_response, organize_into_json, run_all_questions
from src.vectorstore import VectorstoreHandler
from src.models import init_llm

########################################
# Configuration
########################################
SOURCES_DIR = "sources"
JSON_PATH = "multi_source_conversation.json"
AGGREGATION_LLM_NAME = "Llama3.2-3b"

# Dynamically create one RAG config per subfolder under sources
subfolders = [d for d in os.listdir(SOURCES_DIR) if os.path.isdir(os.path.join(SOURCES_DIR, d))]
RAG_CONFIG = {}
for sub in subfolders:
    RAG_CONFIG[sub] = {
        "subfolder": sub,
        "llm_name": "Llama3.2-3b",
        "emb_name": "hf-minilm-l6-v2",
        "k": 10,
    }

handler = VectorstoreHandler(SOURCES_DIR, force_rebuild=False)
rags = {name: create_rag(SOURCES_DIR, cfg, handler) for name, cfg in RAG_CONFIG.items()}
aggregation_llm = init_llm(AGGREGATION_LLM_NAME)


Loading existing vectorstore for sources/book...
Loading existing vectorstore for sources/lectures...


In [3]:

original_prompt = "Explain how the kidneys regulate fluid and electrolyte balance. Include the processes of filtration, reabsorption, secretion, and excretion" 

########################################
# 1) Ask each RAG for initial answer
########################################
initial_responses = generate_answers(original_prompt, rags)

########################################
# 2) Aggregation LLM -> follow-up question for each RAG
########################################
agent_answers = {rag_name: data["answer"] for rag_name, data in initial_responses.items()}
follow_up_questions = generate_follow_up_questions(aggregation_llm, original_prompt, agent_answers)

########################################
# 3) Each RAG answers its follow-up question
########################################
follow_up_responses = {}
for agent_name, question in follow_up_questions.items():
    # Each agent gets exactly one follow-up question
    answers = generate_answers(question, {agent_name: rags[agent_name]})
    follow_up_responses[agent_name] = answers[agent_name]

########################################
# 4) Cross-reference: pick one other RAG to verify each answer
########################################
# We'll do a round-robin: for agent i, use agent (i+1) mod len(rags) to verify


cross_agent_responses = cross_reference(
    rags=rags,
    follow_up_questions=follow_up_questions,
    follow_up_responses=follow_up_responses,
    original_prompt=original_prompt
)

########################################
# 5) Final aggregated answer
########################################
all_responses = {}
for agent_name in rags.keys():
    all_responses[agent_name] = {
        "initial": initial_responses[agent_name]["answer"],
        "follow_up": follow_up_responses[agent_name]["answer"] if agent_name in follow_up_responses else "",
    }

final_answer = aggregate_final_response(aggregation_llm, all_responses, original_prompt)

########################################
# 6) Organize results in JSON and display
########################################
conversation_json = organize_into_json(
    original_prompt,
    initial_responses,
    follow_up_questions,
    follow_up_responses,
    final_answer
)
conversation_json["cross_agent_responses"] = {
    target_rag: {
        verifier: resp.get("answer", "No answer")  # Ensure resp is a dictionary
        for verifier, resp in cross_agent_responses[target_rag].items()
    }
    for target_rag in cross_agent_responses
}

json.dump(conversation_json, open(JSON_PATH, "w"), indent=2)


In [4]:
# Define folder for logs
LOG_FOLDER = "question_logs"
save_folder = LOG_FOLDER + "/easier_questions"
os.makedirs(save_folder, exist_ok=True)
# Example usage:
questions = [
    "How does synaptic plasticity in the hippocampus contribute to memory formation and retrieval?",
    "What is the role of the amygdala in fear conditioning, and how is this studied experimentally?",
    "Describe how dopaminergic pathways underlie reward-related behavior and drug addiction.",
    "Discuss the influence of the prefrontal cortex on decision-making, impulse control, and executive functions.",
    "Explain how chronic stress affects the hypothalamic–pituitary–adrenal (HPA) axis and impacts behavior and cognition."
]
final_qa = run_all_questions(questions, rags, aggregation_llm, save_folder)



In [5]:
save_folder = LOG_FOLDER + "/harder_questions"
os.makedirs(save_folder, exist_ok=True)
questions = [
    "How does synaptic plasticity contribute to the neural basis of learning and memory in behavioral neuroscience?",
    "What role do neuromodulators like dopamine and serotonin play in decision-making and reward-based behaviors?",
    "How can neuroimaging techniques be used to identify the neural correlates of anxiety and stress responses?",
    "What are the behavioral and neural consequences of disrupting the hypothalamic-pituitary-adrenal (HPA) axis?",
    "How do genetic and environmental factors interact to shape individual differences in cognitive and emotional behaviors?"
]

final_qa = run_all_questions(questions, rags, aggregation_llm, save_folder)
